# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [16]:
import pandas as pd
df = pd.read_csv(r'C:\Users\fmrol\Documents\Ironhack\Classes Material\Week 7\LAB  Chains in LangChain\lab-chains-in-langchain\data\Data.csv')

In [17]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [18]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [20]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.2)


In [28]:
prompt = ChatPromptTemplate.from_template(  "Write an ironic product description for {product} in the style of Moira Rose from Schitt's Creek. "
    "Highlight its key features and benefits."
)

In [29]:
# old code 
#chain = LLMChain(llm=llm, prompt=prompt)
#correct code
chain = prompt | llm


In [30]:
product = "Luxury Air Mattress"
chain.invoke(product)

AIMessage(content='Introducing the luxurious Air Mattress fit for a queen, darling. Made from the finest materials known to man, this opulent creation will have you feeling like royalty as you drift off into a peaceful slumber.\n\nKey features include a built-in air pump for effortless inflation, a plush velvety surface that caresses your skin like a gentle breeze, and a reinforced interior that can support even the most regal of sleepers.\n\nBut the real pièce de résistance? The portable design that allows you to bring this majestic mattress with you wherever you go, ensuring that your beauty sleep is never compromised, no matter where your travels take you.\n\nSo go ahead, darling, treat yourself to the finer things in life with our Luxury Air Mattress. Because nothing says glamour like sleeping on a cloud of luxury.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 163, 'prompt_tokens': 37, 'total_tokens': 200, 'completion_tokens_details'

## SimpleSequentialChain

In [31]:
from langchain.chains import SimpleSequentialChain

In [32]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
"Write a description for {product} in the style of Moira Rose from Schitt's Creek. "
    "Highlight its key features and benefits. Be short and concise"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [33]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review {review}, rate the quality of this product from zero to ten in the style of Moira Rose from Schitt's Creek."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [34]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [35]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Indulge in the opulent comfort of this exquisite air mattress, darling. Crafted with premium materials and plush layers, it cradles you in luxury all night long. Experience the unparalleled support and decadent softness as you drift off to dreamland. Elevate your sleep experience with this must-have for the discerning sleeper. It's simply the perfect blend of elegance and relaxation, dear.
I would rate this exquisite air mattress a solid ten, dear! It truly is a luxurious blend of elegance and relaxation, darling. You simply must indulge in the opulent comfort it provides. It's a must-have for the discerning sleeper, sweetie!

> Finished chain.


{'input': 'Luxury Air Mattress',
 'output': "I would rate this exquisite air mattress a solid ten, dear! It truly is a luxurious blend of elegance and relaxation, darling. You simply must indulge in the opulent comfort it provides. It's a must-have for the discerning sleeper, sweetie!"}

**Repeat the above twice for different products**

## SequentialChain

In [72]:
from langchain.chains import SequentialChain

In [73]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into french. "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='fr_review' #Give a name to your output
                    )


In [74]:
second_prompt = ChatPromptTemplate.from_template(
    'Based on {fr_review}, summarize this review in less than 10 words in french.'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='fr_short_review' #give a name to this output
                    )


In [76]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {fr_short_review} into english"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='en_short_review'
                      )


In [77]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"Rate the quality of the translation of {fr_short_review} into {en_short_review} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='rating'
                     )



In [78]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=['fr_short_review','en_short_review', 'rating'],
    verbose=True
)

In [79]:
review = df.Review[5]
overall_chain(review)

C:\Users\fmrol\AppData\Local\Temp\ipykernel_23124\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'fr_short_review': 'Saveur mediocre, mousse pas bonne, possible faux produit.',
 'en_short_review': 'Taste mediocre, mousse not good, possibly fake product.',
 'rating': 'I would rate this translation a 7 out of 10. It effectively conveys the meaning of the original sentence, but could be improved by using more natural language and phrasing.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [100]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [101]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}


In [103]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [104]:
llm = ChatOpenAI(temperature=0)

In [105]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [106]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [108]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [109]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content)

chain = (
    RunnablePassthrough()
    | {
        "input": RunnablePassthrough(),
        "chosen_prompt": router_chain,
    }
    | RunnableLambda(lambda x: prompt_templates[x["chosen_prompt"]].format(input=x["input"]))
    | llm
)



In [113]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")


KeyError: '```json\n{\n    "destination": "math",\n    "next_inputs": "Explain what principle component analysis is using linear algebra"\n}\n```'

In [114]:
chain.invoke("What is black body radiation?")

KeyError: '```json\n{\n    "destination": "physics",\n    "next_inputs": "What is black body radiation?"\n}\n```'

In [115]:
chain.invoke("what is 2 + 2")

KeyError: '```json\n{\n    "destination": "math",\n    "next_inputs": "what is 2 + 2"\n}\n```'

**Repeat the above at least once for different inputs and chains executions - Be creative!**